In [1]:
# automatically reload imported modules
%load_ext autoreload
%autoreload 2

train a model

In [4]:
import helper.get_image
import numpy as np

FOLDER_PATH = 'F://term7//CV//ProjectTrail//A_manualLabel'


# random
train_size = 200
test_size = 100
val_size = 40

img_range = np.arange(1, 800)
X_train_pos_idx, X_test_pos_idx, X_val_pos_idx = helper.get_random_indices(img_range, train_size, test_size, val_size)
X_train_neg_idx, X_test_neg_idx, X_val_neg_idx = helper.get_random_indices(img_range, train_size, test_size, val_size)

X_train, Y_train = helper.get_concrete_data(X_train_pos_idx, X_train_neg_idx, path = FOLDER_PATH)
#X_test , Y_test  = helper.get_concrete_data(X_test_pos_idx, X_test_neg_idx, path = FOLDER_PATH)
#X_val  , Y_val   = helper.get_concrete_data(X_val_pos_idx, X_val_neg_idx, path = FOLDER_PATH)

print( X_train.shape, Y_train.shape )
#print( X_test.shape , Y_test.shape  )
#print( X_val.shape  , Y_val.shape   )

(400, 227, 227) (400,)


In [5]:
from filters import *
bilateral_canny = BilateralCanny()

In [6]:
filters = {}
filters['canny']           = bilateral_canny.canny_img
filters['bilateral canny'] = bilateral_canny.bilateral_canny_img

In [7]:

from preprocessing import shadow_reduction

In [8]:
from sklearn import linear_model

In [9]:
X_train = np.array(list(shadow_reduction.adaptiveThreshold(x) for x in X_train))
tmpX = np.array(list(filters['bilateral canny'](x) for x in X_train)).reshape(2*train_size, -1)
model =  linear_model.LogisticRegression(C=1e5)
model.fit(tmpX, Y_train)

LogisticRegression(C=100000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

## video generation

### video to frames

In [11]:
video_file = "F://term7//CV//ProjectTrail//videos//IMG_5175.mov"
frame_folder = "F://term7//CV//ProjectTrail//IMG_5175frame"
output_frame_folder =  "F:\\term7\\CV\\ProjectTrail\OutputIMG_5175\\"
output_video = "F://term7//CV//ProjectTrail//videos//OutIMG_5175.mov"

from video import video
v = video(video_file)
# v.video_to_frames(frame_folder)

### process to video

In [12]:
 %reload_ext autoreload

In [13]:
import helper.utils
import helper
print(dir(helper))
from helper import rolling_window, shade_area, utils, generate_shadowed_img
import preprocessing.shadow_reduction
import cv2

['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'binary_dilation', 'ceil', 'count', 'cv2', 'dir', 'floor', 'generate_shadowed_img', 'generate_sub_frames', 'get_concrete_data', 'get_image', 'get_images', 'get_random_indices', 'get_stride', 'glob', 'ndimage', 'np', 'os', 'pattern', 'rename', 'rolling_window', 'save_image', 'shade_area', 'sys', 'titlePattern', 'utils']


In [1]:
import time
import os

## CAREFUL!
in the cell below:         frame_final = generate_shadowed_img(frame, frame_after_canny, model)

in the function generate_shadowed_img, model.predict(img) will be called. So for CNN check the predict function again!!!!!

In [19]:
input_format = 'jpg'

bilateral_canny_video = filters['canny']

import time
time_start = time.time()
time_pre = time.time()
time_norm = 0
time_canny =  0
time_predict_and_shadow = 0
time_save = 0

count = 0
frameid = 0
timelog = False

for f in os.listdir(frame_folder):
    if f.endswith(input_format):
        image_path = os.path.join(frame_folder, f)
        frame = cv2.imread(image_path)
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # reduce shadow by dilate, median, or threshold

        if timelog:
            time_pre = time.time()
# shadow reduction
#         frame_shadow_reduced = norm_dilate_median(gray_frame)
        frame_shadow_reduced = shadow_reduction.adaptiveThreshold(gray_frame)
        if timelog:
            time_norm += time.time() - time_pre
            time_pre = time.time()
        
            
#         # generate output after canny filter, or tophat
        frame_after_canny = bilateral_canny_video(frame_shadow_reduced)
#         if timelog:
#             time_canny += time.time() - time_pre
#             time_pre = time.time()    
        
#         frame_final = generate_shadowed_img(frame, frame_after_canny, model)
#         time_predict_and_shadow += time.time() - time_pre
#         time_pre = time.time()
        
        # save to folder
        utils.save_image(cv2.cvtColor(frame_final, cv2.COLOR_RGB2BGR), output_frame_folder, f)
        time_save= time.time() - time_pre
        time_pre = time.time()    
        
if timelog:
    print("total time for frames generation is {}".format(time.time()-time_start))
    print("time norm{}".format(time_norm))
    print("time_canny{}".format(time_canny))
    print("time_predict_and_shadow{}".format(time_predict_and_shadow))
    print("time_save{}".format(time_save))

MemoryError: 

### output to video

In [ ]:
time_start = time.time()
from video import video
v = video(video_file)
v.frames_to_video(output_video, input_loc=output_frame_folder, debug=True)
print("total time video saving {}".format(time.time()-time_start))